Exercise-1
---

Create a class Trellis that
- takes in four arguments: `match_weight`, `delete_weight`, `add_weight`, and `scoring_func`.
    - `scoring_func` is a function that computes the distance or score between two `values`.
    - `match_weight`, `delete_weight`, `add_weight` are floats that weigh a diagonal, horizontal, and vertical transitions, respectively.
- contains a method `match(X, Y)` where X and Y are arrays of `values`; the `values` can be characters, scalars, or even vectors that returns the minimum-edit-distance/matching-score between X and Y and the shortest path (as an array of 2-tuples).

In [81]:
import numpy as np
from copy import deepcopy

class Trellis:
    
    def __init__(self, scoring_func, match_weight=1.0, delete_weight=1.0, add_weight=1.0):
        self.scoring_func = scoring_func
        self.match_weight = match_weight
        self.delete_weight = delete_weight
        self.add_weight = add_weight
    
    def match(self, X, Y, normalize_score=True):
        scoring_func = self.scoring_func
        match_weight, delete_weight, add_weight = self.match_weight, self.delete_weight, self.add_weight
    
        score_rows = np.zeros((2, len(X)+1))
        path_counts = np.zeros((2, len(X)+1))
        back_pointers = []
        for ii in range(len(X)):
            back_pointers.append([])
        score_rows[0, 1:] = 1e9
        score_rows[1:, 0] = 1e9
        
        
        jj = 1
        while jj < len(Y) + 1:
            back_pointer_before_iteration = deepcopy(back_pointers)
            for ii in range(1, len(X)+1):
                diag_score = score_rows[0, ii-1] + match_weight
                vert_score = score_rows[0, ii] + add_weight
                horiz_score = score_rows[1, ii-1] + add_weight
                min_score = min(diag_score, vert_score, horiz_score)
                if min_score == diag_score:
                    back_pointers[ii-1] = list(back_pointer_before_iteration[ii-2])
                    back_pointers[ii-1].append( (jj-2, ii -2) )
                    #print("DIAG", ii-1, back_pointers)
                elif min_score == vert_score:
                    back_pointers[ii-1].append( (jj-2, ii-1) )
                    #print("VERT", ii-1, back_pointers)
                else:
                    back_pointers[ii-1] = list(back_pointers[ii-2])
                    back_pointers[ii-1].append( (jj-1, ii-2) )
                    #print("HORIZ", ii-1, jj-1, back_pointers)
                node_total = min_score + scoring_func(X[ii-1], Y[jj-1])
                score_rows[1, ii] = node_total
            score_rows[0, :] = score_rows[1, :]
            score_rows[1, 1:] = 0
            jj += 1
            #print(back_pointers)
        return score_rows[0, -1], back_pointers[-1]

In [82]:
if __name__=="__main__":
    trellis = Trellis(match_weight=0.0, scoring_func=lambda x, y: 0.0 if x == y else 1.0)

    test_cases = [
        ['TEST', 'TES'],
        ['geek', 'gesek'],
        ['ISLANDER', 'SLANDER'],
        ['MART', 'KARMA'],
        ['TEST', "TEST"]
    ]

    for case in test_cases:
        print(case, trellis.match(case[0], case[1], normalize_score=True)[1])

['TEST', 'TES'] [(-1, -1), (0, 0), (1, 1), (2, 2)]
['geek', 'gesek'] [(-1, -1), (0, 0), (1, 1), (2, 1), (3, 2)]
['ISLANDER', 'SLANDER'] [(-1, -1), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6)]
['MART', 'KARMA'] [(-1, -1), (0, 0), (1, 1), (2, 2), (3, 2)]
['TEST', 'TEST'] [(-1, -1), (0, 0), (1, 1), (2, 2)]


Exercise-2
---

Use the matching algorithm written above to correct spellings of words intput thru the keyboard.
I.e. create your own spell checker! (albiet it being quite inefficient...)

A list of english words has been given to you in `words.txt`

In [83]:
if __name__=="__main__":
    dictionary = list(filter(None, open("words2.txt","r").read().split("\n")))

    trellis = Trellis(lambda x, y: 0.0 if x == y else 1.0, delete_weight=4.0)
    print("Enter /quit to quit")
    while True:
        x = input("word>> ")
        x = x.lower()
        if x == "/quit":
            print("Goodbye!")
            break
        if x in dictionary:
            print("word found")
            continue
        min_sc = 1e9
        match = x
        for el in dictionary:
            sc = trellis.match(el, x, normalize_score=False)[0]
            if sc < min_sc:
                min_sc = sc
                match = el
        print("closest match: ", match)

Enter /quit to quit
word>> hi
word found
word>> /quit
Goodbye!
